# Overview

Produce a method to obtain all assingments for some small $G$.

Where graphs must have vertex 0 up to n-1

We store all ouptuts in the appropriate folder of \Graph\graph_stability_conditions as a text file.

The data is also stored in .pkl format in Stability_Conditions\Graph\find_cycles\examples.

# Table of contents
1. [Finding all weak stability conditions for cases ](#s1)
2. [Functions](#s2)
    1. [Weak_stability_genus_1](#s21)
    2. [Get_assignment_datum Functions](#s22)
    3. [Exhaustive method functions](#s23)
        1. [Initalisation functions](#s231)
        2. [get_Extensions_M_by_T_subg ](#s232)
        3. [Exhuastive Method](#s233)
    4. [After obtaining assignments for G](#s24)

# Finding all weak stability conditions for cases <a name="s1"></a>

## Cyclic graphs: genus 1

In [12]:
graphname="G3"
G=Graph([("0","1"),("1","2"),("2","0")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 2 assignment datums: 
#  2: Produce a stability condition 
#  0: Do NOT produce a stability condition

Length of Assignments I WANT: 3
Number of partial assignments: 2
Length of assignments----------------- 3
Of the 2 assignment datums: 
 2: Produce a stability condition 
 0: Do NOT produce a stability condition


In [19]:
graphname="G4"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 6 assignment datums:
#  6: Produce a stability condition
#  0: Do NOT produce a stability condition

In [13]:
graphname="G5"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 24 assignment datums:
#  24: Produce a stability condition
#  0: Do NOT produce a stability condition

In [14]:
graphname="G6"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 120 assignment datums:
#  120: Produce a stability condition
#  0: Do NOT produce a stability condition

## Genus 2 graphs with middle edges

In [20]:
graphname="G4M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 6 assignment datums:
#  6: Produce a stability condition
#  0: Do NOT produce a stability condition

In [12]:
graphname="G5M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 24 assignment datums:
#  24: Produce a stability condition
#  0: Do NOT produce a stability condition

Length of Assignments I WANT: 11
Number of partial assignments: 144
Length of assignments----------------- 8
13:144 Progress: 10%
14:144 Progress: 10%
28:144 Progress: 20%
42:144 Progress: 30%
56:144 Progress: 40%
57:144 Progress: 40%
71:144 Progress: 50%
85:144 Progress: 60%
86:144 Progress: 60%
100:144 Progress: 70%
114:144 Progress: 80%
128:144 Progress: 90%
129:144 Progress: 90%
Number of partial assignments: 40
Length of assignments----------------- 9
3:40 Progress: 10%
7:40 Progress: 20%
11:40 Progress: 30%
15:40 Progress: 40%
19:40 Progress: 50%
23:40 Progress: 60%
27:40 Progress: 70%
31:40 Progress: 80%
35:40 Progress: 90%
Number of partial assignments: 24
Length of assignments----------------- 10
11:24 Progress: 50%
Number of partial assignments: 24
Length of assignments----------------- 11
Of the 24 assignment datums: 
 24: Produce a stability condition 
 0: Do NOT produce a stability condition


In [13]:
graphname="G6M2"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 120 assignment datums:
#  120: Produce a stability condition
#  0: Do NOT produce a stability condition

In [14]:
graphname="G6M3"  
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 588 assignment datums: 
#  120: Produce a stability condition 
#  468: Do NOT produce a stability condition

In [ ]:
graphname="G8M04"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","7"),("7","0"),("0","4")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)



In [ ]:
graphname="G8M02"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","7"),("7","0"),("0","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)



## Vine blowups

In [12]:
graphname="V_222"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","4"),("4","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

Length of Assignments I WANT: 12
Number of partial assignments: 36
Length of assignments----------------- 7
17:36 Progress: 50%
Number of partial assignments: 28
Length of assignments----------------- 9
13:28 Progress: 50%
Number of partial assignments: 44
Length of assignments----------------- 11
8:44 Progress: 20%
12:44 Progress: 30%
21:44 Progress: 50%
30:44 Progress: 70%
34:44 Progress: 80%
Number of partial assignments: 38
Length of assignments----------------- 12
Of the 38 assignment datums: 
 38: Produce a stability condition 
 0: Do NOT produce a stability condition


In [56]:
graphname="V_223"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","5"),("5","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

In [ ]:
graphname="V_224"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

In [12]:
graphname="V_233"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

Length of Assignments I WANT: 21
Number of partial assignments: 576
Length of assignments----------------- 9
0:576 Progress: 0%
1:576 Progress: 0%
54:576 Progress: 10%
55:576 Progress: 10%
56:576 Progress: 10%
57:576 Progress: 10%
58:576 Progress: 10%
59:576 Progress: 10%
112:576 Progress: 20%
113:576 Progress: 20%
114:576 Progress: 20%
115:576 Progress: 20%
116:576 Progress: 20%
117:576 Progress: 20%
169:576 Progress: 30%
170:576 Progress: 30%
171:576 Progress: 30%
172:576 Progress: 30%
173:576 Progress: 30%
174:576 Progress: 30%
227:576 Progress: 40%
228:576 Progress: 40%
229:576 Progress: 40%
230:576 Progress: 40%
231:576 Progress: 40%
232:576 Progress: 40%
285:576 Progress: 50%
286:576 Progress: 50%
287:576 Progress: 50%
288:576 Progress: 50%
289:576 Progress: 50%
342:576 Progress: 60%
343:576 Progress: 60%
344:576 Progress: 60%
345:576 Progress: 60%
346:576 Progress: 60%
347:576 Progress: 60%
400:576 Progress: 70%
401:576 Progress: 70%
402:576 Progress: 70%
403:576 Progress: 70%
4

In [ ]:
graphname="V_225"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

Length of Assignments I WANT: 24
Number of partial assignments: 4320
Length of assignments----------------- 10
0:4320 Progress: 0%
1:4320 Progress: 0%
2:4320 Progress: 0%
3:4320 Progress: 0%
4:4320 Progress: 0%
5:4320 Progress: 0%
6:4320 Progress: 0%
7:4320 Progress: 0%
8:4320 Progress: 0%
9:4320 Progress: 0%
10:4320 Progress: 0%
11:4320 Progress: 0%
12:4320 Progress: 0%
13:4320 Progress: 0%
14:4320 Progress: 0%
15:4320 Progress: 0%
16:4320 Progress: 0%
17:4320 Progress: 0%
18:4320 Progress: 0%
19:4320 Progress: 0%
20:4320 Progress: 0%
410:4320 Progress: 10%
411:4320 Progress: 10%
412:4320 Progress: 10%
413:4320 Progress: 10%
414:4320 Progress: 10%
415:4320 Progress: 10%
416:4320 Progress: 10%
417:4320 Progress: 10%
418:4320 Progress: 10%
419:4320 Progress: 10%
420:4320 Progress: 10%
421:4320 Progress: 10%
422:4320 Progress: 10%
423:4320 Progress: 10%
424:4320 Progress: 10%
425:4320 Progress: 10%
426:4320 Progress: 10%
427:4320 Progress: 10%
428:4320 Progress: 10%
429:4320 Progress: 10

In [ ]:
graphname="V_234"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

In [ ]:
graphname="V_235"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","8"),("8","2")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

In [ ]:
graphname="V_333"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","0"),("0","6"),("6","7"),("7","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

In [ ]:
graphname="V_334"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","5"),("5","6"),("6","0"),("0","7"),("7","8"),("8","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

## Higher genus graphs

In [21]:
graphname="G4M02M13"
G=Graph([("0","1"),("1","2"),("2","3"),("3","0"),("0","2"),("1","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 28 assignment datums: 
#  10: Produce a stability condition 
#  18: Do NOT produce a stability condition

In [22]:
graphname="G5M02M03"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

# Of the 68 assignment datums: 
#  24: Produce a stability condition 
#  44: Do NOT produce a stability condition

In [23]:
graphname="G5M02M03M14"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4")], multiedges=True)
ordering_trees=list(G.spanning_trees())

T=ordering_trees[2]

M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

#testing time it takes if chose anothe starting tree.

# tree;length of assignments at start

# 0:11
# 1:17
# 2:
    
# T=ordering_trees[2]
#     Of the 1034 assignment datums:
#  82: Produce a stability condition
#  952: Do NOT produce a stability condition

In [ ]:
graphname="G5M02M03M14M13"
G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4"),("1","3")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0]
M=get_all_assingment_datum(G,T)
assignment_datum=condition_checker(M)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)



## Unknown examples: Find size of stabilities of K5 and K33

In [ ]:
# graphname="K5"
# G=Graph([("0","1"),("1","2"),("2","3"),("3","4"),("4","0"),("0","2"),("0","3"),("1","4"),("1","3"),("2","4")], multiedges=True)
# ordering_trees=list(G.spanning_trees())
# T=ordering_trees[0]
# M=get_all_assingment_datum(G,T)
# assignment_datum=condition_checker(M)
# store_data(G,assignment_datum,ordering_trees,graphname)
# pickle_assignment_datum(graphname,assignment_datum)

In [ ]:
# graphname="K33"
# G=Graph([("0", "3"), ("0", "4"), ("0", "5"), ("1", "3"), ("1", "4"), ("1", "5"), ("2", "3"), ("2", "4"), ("2", "5")], multiedges=True)
# ordering_trees=list(G.spanning_trees())
# T=ordering_trees[61] #Choosing a tree of I6, tree of I6.
# M=get_all_assingment_datum(G,T)
# assignment_datum=condition_checker(M)
# store_data(G,assignment_datum,ordering_trees,graphname)
# pickle_assignment_datum(graphname,assignment_datum)

In [43]:
graphname="K33_breaks"
G=Graph([("0", "3"), ("0", "4"), ("0", "5"), ("1", "3"), ("1", "4"), ("1", "5"), ("2", "3"), ("2", "4"), ("2", "5")], multiedges=True)
ordering_trees=list(G.spanning_trees())
T=ordering_trees[0] #Choosing a tree of I6, tree of I6.
# M=get_all_assingment_datum(G,T)
assign=[[(tree,[np.array([0]*6)]) for tree in ordering_trees]]
assignment_datum=condition_checker(assign)
store_data(G,assignment_datum,ordering_trees,graphname)
pickle_assignment_datum(graphname,assignment_datum)

Of the 1 assignment datums: 
 1: Produce a stability condition 
 0: Do NOT produce a stability condition


# Functions <a name="s2"></a>

## Weak_stability_genus_1 <a name="s21"></a>

In [6]:
#For change_to_get_trees, produces stability conditions for genus 1 graph given by a cycle.
def head(n, prev_tree_edge):
    """
    n= number of vertices in I_n
    previous element of the cycle, ie tree_edge is j where e_j ={v_j,v_{j+1}} 
    
    #Also used in chip adding on necklace curve
    """
    head = np.zeros(n)

    if prev_tree_edge == n:
        head[0] = 1
    else:
        head[prev_tree_edge] = 1

    return head
def tail(n, current_tree_edge):
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = -1

    return tail
def tail_plus(n, current_tree_edge):  #for chip adding
    """
    n= number of vertices in I_n
    tree_edge is j where e_j ={v_j,v_{j+1}}
    i is previous element of the cycle, ie tree_edge
    """
    tail = np.zeros(n)
    # if current_tree_edge==n:
    #     tail[0]=-1
    tail[current_tree_edge - 1] = 1

    return tail

def cycle_to_assignments(cycle):
    #Main to get assingments from cycle

    n = len(cycle)

    # cycle=(1,3,4,2)
    ass_0 = np.zeros(n)

    M = [(1, ass_0)]  # Memory

    for i in range(1, len(cycle)):

        current_tree_edge = cycle[i]
        tup = M[-1]  #pick last item of M

        vtail = tail(n, current_tree_edge)
        vhead = head(n, tup[0])
        prev_ass = tup[1]

        current_ass_t = prev_ass + vtail + vhead
        M.append((current_tree_edge, current_ass_t))

    # for i in M:
    #     print(i)

    return M
def change_to_get_trees(cycle):
    
    #return trees where there was previously just numbers indicating edge to be removed.
        # change: [(1, array([0., 0., 0., 0.])), (3, array([ 0.,  1., -1.,  0.])), (2, array([ 0.,  0., -1.,  1.])), (4, array([0., 0., 0., 0.]))]
    # so first term is tree of In
    # cycle=(1,3,2,4)

    data_In=cycle_to_assignments(cycle) #<-----------------
    data_In=sorted(data_In, key=lambda x: x[0]) # reordering to (1,),(2,) ect.. instead of being ordered by cycle.
    
    """
    Wish to order data_In by first term of tuple so that is (1, ),(2,) thus putting trees in order so that when we append to Memory M we have
    M'[counter](0) = M[counter][0] when we are iterating through counters.
    """
    
    n=len(cycle)
    G=graphs.CycleGraph(n)
    
    new_data_In=[]
    for data in data_In:
        #Get tree
        tree_info=data[0]
    #     print(tree_info)
        if tree_info== len(data_In):
            edge_tree=(tree_info-1,0)
        else:
            edge_tree=(tree_info-1,tree_info)
    #     print(edge_tree)
        tree=copy(G)
        tree.delete_edge(edge_tree)

        #Recompile data
        new_data_In.append((tree,data[1]))

    # plot(new_data_In[2][0]) #Have the correct trees.
    
    """
    Example:x=change_to_get_trees(cycle)

    print(cycle)
    for i in x:
        print(i[0].edges())
        
        cycle=(1,3,2)
    x=change_to_get_trees(cycle)
    print([tup[0].edges() for tup in x])
    """
    
    return new_data_In

def get_cycles(n):
    lst = list(range(1, n + 1))
    x = permutations(lst, n)
    y = [i for i in x if i[0] == 1]

    #Does:
    # x=permutations([1,2,3,4,5],5)
    # y=[i for i in x if i[0]==1]
    # # print(len(y))
    # for i in y:
    #     print(i)

    return y

In [8]:
# l = [head,tail,tail_plus,cycle_to_assignments,change_to_get_trees,get_cycles]
# find_functions_used_l(l)

## Get_assignment_datum Functions <a name="s22"></a>

In [9]:
#For get_all_assingment_datum: produces all assignments for G.
def get_subgraphs_gm1_Edgesmissing(G):
    
    #Get all spanning subgraphs, on row above trees specifically, $\tilde{\Gamma}_{g-1}$.
    #returns a list of such subg
    
    edge_num_G=len(G.edges())
    vert_num_G=len(G.vertices())
    g=edge_num_G-vert_num_G+1
    gm1=g-1

    E=Set(G.edges()) #remeber possibly mulitgraph
    V=G.vertices()

    graphs_i_want=[]
    for s in E.subsets():
        if len(s)==edge_num_G-gm1: # edge_num_G-gm1number of edges of these subgraphs.
            H=Graph(multiedges=True)
            H.add_edges( s )
            H.add_vertices( V )        # are interested only in spanning subgraphs
            if H.is_connected(): #Want connected subgraphs
                graphs_i_want.append(H)

    return graphs_i_want

def check_progress(i, k): #for get_all_assingment_datum
    progress = (i + 1) / k * 100  # calculate the progress as a percentage
    if round(progress) % 10 == 0 and round(progress) != 100:  # check if progress is at 10%, 20%, ..., 90%
        print(f"{i}:{k} Progress: {round(progress)}%")
    return   

def get_all_assingment_datum(G,T,starting_ass=None): #draft2
    
    """
    Returns:
        a list of all assignment datum to be tested for size.
    # Inputs:
        # G #main graph
        # T #fixed tree with assignment 0
        #start_ass= is a np.array([a,b,..]) of length the number of vertices of G if none then takes all 0 assignment.
    """
    
    #Initalisation:
    vert_num_G=len(G.vertices()) #subg is spanning so all verts of G
    if starting_ass==None:
        starting_ass=np.array([0]*vert_num_G)
    subgraphs= get_subgraphs_gm1_Edgesmissing(G) #\tilde{G} the subgraphs T\cup e.
    k_G=len(list(G.spanning_trees()))# number of spanning trees
    M=[(T,[starting_ass])] #Memory <--- Starting case

    print("Length of Assignments I WANT:",k_G)

    #Main loop
    "We now work to obtain the List of all assignments"
    partial_all_assignments=[M] #Main object we want to build, with starting case.
    
    for counter1 in range(k_G):# Runs over spanning trees to which we have assignments for, until $|M|=k(\Gamma)$.        
        updated_partial_all_assignments=[]
        
        for ind,M in enumerate(partial_all_assignments):
            
            ######## TESTING ############################################################
            #For when taking long: to measure
            check_progress(ind, len(partial_all_assignments))   
            ######## TESTING ############################################################

            """we now obtain a list of all assignments obtained by extending assignment M by searching through subg containing T
            for different cycles."""
            Extensions_M_by_T=Exhaustive_method(M,counter1,subgraphs) # A list of list=[(T,ass_t)| for some subset of trees]
            
            updated_partial_all_assignments.extend(Extensions_M_by_T)
        
#         """We now update partial_all_assignments to updated_partial_all_assignments, which have assignments on new trees contained in subg containing T."""
        partial_all_assignments=updated_partial_all_assignments
        
        print("Number of partial assignments:",len(partial_all_assignments))
        print("Length of assignments-----------------",len(partial_all_assignments[0]))
        
        """Once we have all trees in the assingment we are done"""
        if len(partial_all_assignments[0])==k_G: #it is enough to check the first element as number of extra trees added is constant
            break             

    return partial_all_assignments

In [10]:
# l = [get_subgraphs_gm1_Edgesmissing,check_progress,get_all_assingment_datum]
# find_functions_used_l(l)

get_subgraphs_gm1_Edgesmissing:
{'len', 'Set', 'Graph'} 

check_progress:
{'round', 'print'} 

get_all_assingment_datum:
{'range', 'enumerate', 'list', 'len', 'get_subgraphs_gm1_Edgesmissing', 'Exhaustive_method', 'check_progress', 'print'} 



## Exhaustive method functions <a name="s23"></a>

The function Exhaustive method is composed of parts.
These are chrologically given below.

### Initalisation functions <a name="s231"></a>

In [12]:
#for subg in subg_l_contain_T: loop
def get_tails(subg):
    #tails = edges with vertices outside of subnecklace. 
    #return multigraph of tails after removing edges of subnecklace.

    subg_edges=subg.edges()
    #Subnecklace edges
    [edges_subnecklace]=subg.cycle_basis(output='edge')

    #Get tails
    tails_subg=copy(subg)
    tails_subg.delete_edges(edges_subnecklace) #order of the edges must matter.

    return tails_subg

def skipping_subg_we_know(subg,pre_M):
    #Skipping subg if its trees are already in pre_M
    check=all(tree in [tup[0] for tup in pre_M] for tree in list(subg.spanning_trees()))
    return check

In [13]:
# x=[get_tails,skipping_subg_we_know]
# find_functions_used_l(x)

get_tails:
{'copy'} 

skipping_subg_we_know:
{'list', 'all'} 



### get_Extensions_M_by_T_subg <a name="s232"></a>

In [14]:
def get_Extensions_M_by_T_subg(pre_M,common_tree,common_tree_assingment,subg,n,tails):
    """
    Returns: a list of all (different) extensions of M (a list of tuples) (used to obtain Extensions_M_by_T) compatible with T=M[counter][0] iterating over a single subgraph. 
    By considering all cycles that work with M and labeling of In both ways.

    Returns a list of objects the form:
    [(T,Ass_T) for pre_M, (T,Ass_T) for cycle where we take cycles that work]
    and [(T,Ass_T) for pre_M, (T,Ass_T) for cycle where we take cycles that work] for taking the oppositie labeling for In.
    """

    Extensions_M_by_T_subg=[]
    relabler=get_relabler(subg,common_tree)
    valid_M1=running_cycles(pre_M,n,relabler,tails,common_tree_assingment,change_relabler=False)
    if valid_M1!=None:
        Extensions_M_by_T_subg.extend(valid_M1) #valid_M1 is a list

    return Extensions_M_by_T_subg

get_Extensions_M_by_T_subg:
{'get_relabler', 'running_cycles'} 



#### For running_cycles:

In [16]:
def put_assignments_together(M, new_data): #for append_to_list
    """
    Returns: A list of (T,Ass_T) where Ass_T may have duplicates.
    """
    
    mapping = {t.copy(immutable=True): ass_l for (t, ass_l) in M} #ass_l is a list
    for (t_dif, ass) in new_data:
        #if have same tree in new data, record assignments by adding to mapping
        if t_dif.copy(immutable=True) in mapping:
            mapping[t_dif.copy(immutable=True)] = mapping[t_dif.copy(immutable=True)] + [ass] #mapping[t] and  ass will be list of assingments.
        # If have tree not originally in mapping, add key,value pair.
        else:
            mapping[t_dif.copy(immutable=True)] = [ass]
    
    #Put mapping into list format
    ass_gamma_all=[(t.copy(immutable=False), ass_l) for t, ass_l in mapping.items()] #possible duplicates in list ass
    """I have changed tree back to mutable objects."""
    # """mapping.items() does not sort the lists see change_to_trees."""
    return ass_gamma_all
def selector(l,k): #for append_to_list
    #Returns the tup of l (a list of 2-tuples) for which tup[0] in k
    t=[]
    for tup in l:
        if tup[0] in k:
            t.append(tup)
    return t
def remove_duplicates(ass_gamma_all): #for append_to_list
    #removing duplicates assingments (np.arrays()) treats the list as a set.
    ass_gamma_uni=[] 
    for pair in ass_gamma_all:
        l_unique, index = np.unique(pair[1], return_index=True, axis=0)
        rows=[np.array(row) for row in l_unique]
        ass_gamma_uni.append((pair[0],rows))
    return ass_gamma_uni

def append_to_list(M, new_data,change_relabler): # For running_cycles:
    """
    Inputs:new_data a list of (T,assT) for subg trees and cycle.
    Returns
    """
    #Glue together both lists of data.
    ass_gamma_all=put_assignments_together(M, new_data)
    
    """
    We now need to put the (t,Ass_T) into the order of tree for M + new_data respectiting common trees.
    """
    M_trees=[m[0] for m in M]
    new_data_trees=[m[0] for m in new_data] #All trees in new data including common_tree.
    
    #We remove trees that are in M_tree from new_data_trees.
    common_trees =[item for item in M_trees if item in new_data_trees] #trees that are common to both.
    new_data_trees_not_common_trees= [tree for tree in new_data_trees if tree not in common_trees]

    """We take the part of ass_gamma_all that has trees in M_trees and then order them according to M_trees.
    Similar for new_data_trees_not_common_tree"""
    
    M_part_ass_gamma_all=selector(ass_gamma_all,M_trees)        
    new_data_part_ass_gamma_all=selector(ass_gamma_all,new_data_trees_not_common_trees)
    
    """ISSUE:
    From new_data_part_ass_gamma_all want to remove the tuple for trees we have already in M_trees.
    see: new_data_trees_not_common_tree above.
    """
    
    """It is important we have the correct order for assignments of trees. In particular new_data."""
    
    ass_gamma_all=M_part_ass_gamma_all+new_data_part_ass_gamma_all
    ass_gamma_uni=remove_duplicates(ass_gamma_all)
    return ass_gamma_uni
def check_tuple_size(lst): # For running_cycles
    """
     "Check if, for $M$ we have $|Ass(T)|=1$ for all $T$."
    """
    for tup in lst:
        if len(tup[1]) != 1:
            return False
    return True

def running_cycles(M,n,relabler,tails,common_tree_assingment,change_relabler): # For Exhaustive method function.

    """
    Obj: Run through all cycles for this subnecklace (genus $1$ method to calculate assignments)
    Then compare these new assignments (for trees in subg) to previous assignment datum M and check size is 1. 
    Returns:A list the extensions of an assignment M, for a given subgraph.
    """
    valid_M=[]
    for cycle in get_cycles(int(n)):

        data_subg=get_ass_trees_subg(cycle,relabler,tails,common_tree_assingment)

        """append data_subg to $M$ Increasing the size of $|Ass(T^{'})|$ if necessary. 
        The below code may (see append_to_list) (Think this is fixed now) inadvertantly result in M[counter][0] \ne M'[counter][0] see change_to_get_trees. 
        We only appends to assignments data for trees which we knoe in M and keep tree same, we append new (T,ass_T)."""
        
        if change_relabler==True:
            result = append_to_list(copy(M), data_subg,change_relabler=True) 
        else:
            result = append_to_list(copy(M), data_subg,change_relabler=False) 

        "Check if, for $M$ we have $|Ass(T)|=1$ for all $T$."
        if check_tuple_size(result)==True:
            valid_M.append(result)
            
    if len(valid_M)>0:
        return valid_M 
    
    return None

In [18]:
# x=[put_assignments_together,append_to_list,remove_duplicates,selector,running_cycles]
# find_functions_used_l(x)

##### For get_ass_trees_subg:

In [19]:
def find_leaves(spanning_tree):#For get_ass_trees_subg
    """
    Obj: to get relabeler, need to define a start and end for the path 
    in the subnecklace of of subg wrt common_tree (specifically fixed)
    where spanning_tree = In_common_tree
    #returns the leaves of tree in subnecklace.i.e [start,end]
    """
    nodes=spanning_tree.vertices()
    leaves = []
    for node in nodes:
        edges = [x for x in spanning_tree.edges() if node in x]
        if len(edges) == 1:
            leaves.append(node)
    return leaves
def get_relabler(subg,common_tree,leave_order=None):#For get_ass_trees_subg
    """
    Obj: Get relabling, that is given the path in subnecklace we need a method to 
    relabel the vertices of trees of I_n to the appropriate ones in the subnecklace of sugb.to get relabling we use
    common tree in subg (acting as 0 assignment).
    #returns a dictionary mapping the vertices of the subnecklace to In. 
    
    The specific choose of leave_order does not mather.
    
    Returns a mapping of vertices of In to vertices of subnecklace.
    i.e a dictionary for mapping.
    """
    
    #remove tails from common_tree.
    tails=get_tails(subg) #tails
    In_common_tree=copy(common_tree) #with labeling of subg
    In_common_tree.delete_edges(tails.edges()) #to put into subnecklace form.
    
    [start,end]=find_leaves(In_common_tree)
    [path]=In_common_tree.all_paths(start, end) #of the form [[0, 5, 1, 4, 2, 3]]

    relabler={}    
    for i in range(len(path)):
        """
        The End of the subnecklace should map to v_0 of In
        The Start of the subnecklace should be v_1.
        """
        relabler[i]=path[i-1]
    return relabler
def get_temp_data(relabler,data):#For get_ass_trees_subg
    """
    Obj:Apply relabler: to get trees of subgnecklace from trees of In
    Returns:data but with trees relabled.
    """
    temp_data =[] # temp_data for subnecklace.
    for d in data: #data of In assignments (tree,ass) pairs
        tree_In=copy(d[0])
        tree_In.relabel(relabler)
        temp_data.append((tree_In,d[1]))
    return temp_data
def extend_trees_subnecklace_to_tails(temp_data,tails):#For get_ass_trees_subg
    
    """
    Obj: Extend the tree of new_data to tails only (dont bother assingments yet).
    """
    extendtails_temp_data =[] # temp_data for subnecklace.
    for d in temp_data: #data of In assignments (tree,ass) pairs
        tree_In=copy(d[0])
        tree_In.add_edges(tails.edges())
        extendtails_temp_data.append((tree_In,d[1]))
    return extendtails_temp_data

def build_list(l, relabeling,vert_num_G):#For Extend_relabel_assignments
    

    """Obj: Given a assignment on a tree of In with the relabling relabel_dict={0:4,1:0,2:5,3:1}
    we give the assoicated assignment on the subnecklace of subg, where we put 0's on the tails.
    
    More explicitly:
    - By changing these assignments to ones on the correct vertices of subg using relabel_dict
    - Then extend temp_ass to include the vertices of the tails of subg with value (0).
    
    Inputs:    
        # l #assignment on tree from In
        # relabel_dict #from subgraph
    Returns: Non-tranlated assignment on T in G.
    """
    
    result = [0] * vert_num_G # number of vertices of $G$.

    for i, value in enumerate(l):
        result[int(relabeling[i])] = value
    return result
def Extend_relabel_assignments(data,common_tree_ass,relabler):#For get_ass_trees_subg
    
    """
    Obj:  Relabel,Extend to tail vertices and Tranlsated $Ass(T^{'})$ by $Ass(T)$ for common_tree.
    where  extending and relabeling ass for In to tempoary ass for G.
    Returns: a [(T,Ass_T) | .. ] where T is a tree of G.
    """
    vert_num_G=len(common_tree_ass)
    temp_data=[]
    for d in data:
        assignment=d[1]
        temp_ass=build_list(assignment, relabler,vert_num_G)
        temp_data.append((d[0],temp_ass))
    
    #Now translate by the assingmnet of the common tree.
    final_data= [(tree, arr + common_tree_ass) for tree, arr in temp_data]
    return final_data

def get_ass_trees_subg(cycle,relabler,tails,common_tree_assingment): #For running_cycles
    
    """
    #Obj: Using a cycle to get all tranlsated $Ass(T^{'})$ for all $T^{'}$ in a specific subg.
    #return: list of $(T^{'},Ass(T^{'}))$ for subg
    """
    data=change_to_get_trees(cycle) #get assingment data for In graph first

    #relabel tree of In to trees of subnecklace of subg.
    temp_data=get_temp_data(relabler,data)

    """For the trees we add back the tails of the subgraph relative to the subnecklace."""
    #Now extend the tree of temp_data to tails only (dont bother assingments yet).
    extendtails_temp_data=extend_trees_subnecklace_to_tails(temp_data,tails)

    """#6) Extend and Tranlsated $Ass(T^{'})$ by $Ass(T)$ (a generalisation of $\theta_{i}$ in genus $2$)"""
    final_data=Extend_relabel_assignments(extendtails_temp_data,common_tree_assingment,relabler)
    data_subg=final_data #list of data for subg that we wish to glue onto M

    return data_subg

In [20]:
# x=[find_leaves,Extend_relabel_assignments,extend_trees_subnecklace_to_tails,get_temp_data,get_ass_trees_subg]
# find_functions_used_l(x)

find_leaves:
{'len'} 

Extend_relabel_assignments:
{'len', 'build_list'} 

extend_trees_subnecklace_to_tails:
{'copy'} 

get_temp_data:
{'copy'} 

get_ass_trees_subg:
{'extend_trees_subnecklace_to_tails', 'change_to_get_trees', 'Extend_relabel_assignments', 'get_temp_data'} 



### Exhuastive Method <a name="s233"></a>

In [21]:
def Exhaustive_method(M,counter1,subgraphs):#draft2
    
    """
    #Main function for get_all_assingment_datum.
    
    Inputs: M is a list of tuples (T,Ass_T) ie a partial_assignment M.
    Returns: All extensions of M (Extensions_M_by_T) compatible with T=M[counter][0] iterating over all subgraphs.
    i.e. A list of lists of the form [(T,ass_T)| for some subset of T's]
    """
    
    #Initialisation:
    pair=M[counter1] #some (T,ass_t), will move along M as counter increases.
    common_tree=pair[0]
    [common_tree_assingment]=pair[1] #this is a list,ass there can be multiple assingmnets so need to unpack
    subg_l_contain_T=[subg for subg in subgraphs if common_tree.is_subgraph(subg, induced=False)]   # subgraphs that contain T.

    """We now build the extensions of M per subgraph"""

    pre_Extensions_M_by_T=[M] #Initial case for building updated_pre_Extensions_M_by_T
    for subg in subg_l_contain_T:
        #Initailisation:
        vert_num_G=len(list(subg.vertices()))
        tails=get_tails(subg)
        [edges_subnecklace]=subg.cycle_basis(output='edge') #return cycle of subgnecklace
        n=len(edges_subnecklace)#length of subnecklace
        
        if len(pre_Extensions_M_by_T)==0:
            break
        if skipping_subg_we_know(subg,pre_Extensions_M_by_T[0]): #if we already know the assingments for all trees in subg
            continue

        updated_pre_Extensions_M_by_T=[]
        for pre_M in pre_Extensions_M_by_T:

            Extensions_M_by_T_subg=get_Extensions_M_by_T_subg(pre_M,common_tree,common_tree_assingment,subg,n,tails)
            updated_pre_Extensions_M_by_T=updated_pre_Extensions_M_by_T+Extensions_M_by_T_subg

        pre_Extensions_M_by_T=updated_pre_Extensions_M_by_T #Overriding existing, so have updated pre_Extensions_M_by_T for next subgraph.

    """After iterating through all subgraphs we have the list of all extensions of M from T."""
    Extensions_M_by_T=pre_Extensions_M_by_T 

#     ######### TESTING ##################################################################
#     #Take a random element, trying to find a example that works.   
#     if len(pre_Extensions_M_by_T)>1:
#         random_index = random.randint(0, len(pre_Extensions_M_by_T) - 1)
#         random_element = pre_Extensions_M_by_T[random_index]
#         Extensions_M_by_T=[random_element]
#     ######### TESTING ##################################################################

    
    return Extensions_M_by_T

In [22]:
x=[Exhaustive_method]
find_functions_used_l(x)

Exhaustive_method:
{'get_tails', 'list', 'len', 'get_Extensions_M_by_T_subg', 'skipping_subg_we_know'} 



## After obtaining assignments for G <a name="s24"></a>

In [23]:
# Assignment_datum_for_G for w_stability:
def get_sp_trees(G): #Out: list of edges in spanning tree
    all_trees=[]
    for g in list(G.spanning_trees()):
        all_trees.append(g.edges(sort=True, labels=False))
    return all_trees
def list_difference(a, b):
    "https://stackoverflow.com/questions/8106227/difference-between-two-lists-with-duplicates-in-python"
    count = Counter(a) # count items in a
    count.subtract(b)  # subtract items that are in b
    diff = []
    for x in a:
        if count[x] > 0:
           count[x] -= 1
           diff.append(x)
    return diff

def chip_adding(graph,tree,ass):
    """
    Inputs:
    graph: G_edges graph edges
    tree : spanning tree of graph
    ass : an n=vert(graph) tuple on the spanning tree.
    
    recursion idea: https://stackoverflow.com/questions/53638816/python-library-function-to-re-apply-a-function-to-its-own-output-until-output-re
    """
    
    edges=graph # just want the edges 
    n=len(ass)
    complement=list_difference(graph,tree) #edges in complement.
    
    def rec_funct(edge_l,inputs,n):
        data=[]
        edge=edge_l[0]
        
        for b in inputs:
            delta_1=np.zeros(n)
            delta_2=np.zeros(n)
            
            delta_1[int(edge[0])]=1 #we chip add at the first vertex. #### <- CHANGED as taking vertices 0,..,n-1 now
            delta_2[int(edge[1])]=1 #we chip add at the second vertex.
            
            b1=b+delta_1
            b2=b+delta_2
            data=data+[b1,b2]
        
        return (edge_l,data,n)
    
    def recursion(edge_l,inputs,n):
        
        new_data=rec_funct(edge_l,inputs,n)
        edge_l=edge_l[1:] #removeing first edge        
        new_data=(edge_l,new_data[1],new_data[2])
                
        if len(edge_l)==0:
            new_inputs=new_data[1]
            return new_inputs
        else:
            return recursion(*new_data)
        
    breaks=recursion(complement,[np.zeros(n)],n)
    breaks=np.unique(breaks, axis=0)
    
    patch=[]
    for bbreak in breaks:
        patch.append(ass+bbreak)
    
    return patch

def w_stability(graph,Assignments):
    
    """
    #Graphs are labelled v0 to v_n-1
    Inputs:
        graph: Any smallish finite multigraph
        Assignments: A list of (complexity of graph many) lists of length vert(graph).
    
    Outputs: a list of lists of length vert(graph) corresponding to liine bundle multidegrees obtained by chip adding.

    """
    G=graph
    
    G_edges=G.edges(sort=True, labels=False)
    tree_l=get_sp_trees(G)

    lbm_patches=[] # the set of patches,which we'll take the union of. 
    for index,tree in enumerate(tree_l):    
        ass=Assignments[index]    
        patch=chip_adding(G_edges,tree,ass)  
        
#         print("Patch",[x.astype(int).tolist() for x in patch],"\n") #If want to see patches of lbm.
        
        lbm_patches.append(patch)
    
    sig=np.concatenate(lbm_patches, axis=0)
    
    sig=sig.astype('int32')
    
#     print(np.array(sig))
    
    
    sig=np.unique(sig, axis=0)

    return sig
def check_size(G,sigma):
    if len(sigma)==len(get_sp_trees(G)):
        x="This choice of assignments -- gives -- a stability condition."
        y=True
    else: 
        x="This choice of assignments -- not give -- a stability condition."
        y= False 
    return y,x

In [29]:
# x=[get_sp_trees,w_stability,chip_adding,list_difference,check_size]
# find_functions_used_l(x)

In [25]:
#Given a Assignment_datum_for_G 
def analyse_specific_assignment(N):
    #Returns assignments and trees.
    for pair in N:
        print(f"tree: {pair[0].edges()}")
        [assingment_at_tree]=pair[1]
        print(assingment_at_tree.tolist())
        print("\n")      
    return

def get_ordered_list(M, ordering_trees):# for get_assignment_ordered_for_wstab

    #Given M produce a list of assignments whose ordering is dependant on a specified ordering of trees ordering_trees.
    #M: memory for storing pairs (T,Ass_T).
    #returns list of assignments ordered wrt ordering_trees. 
    P = []
    mapping = {t.copy(immutable=True): ass for (t, ass) in M}
    for t in ordering_trees:
        P.append(mapping[t.copy(immutable=True)])
    return P

#After obtaining assignments: Putting into the correct format in order to apply w_stability

def get_assignment_ordered_for_wstab(M,ordering_trees):
    #produce a list of assignments (only) whose ordering is dependant on a specified ordering of trees ordering_trees.
    #Puts into correct form
    N = get_ordered_list(M, ordering_trees) 
    N=[list(x) for x in N] #turn each on arrays to lists.
#     then put in the correct form:
    result = [] 
    for elem in N:
        l=elem[0].astype(int)
        result.append(l.tolist())
    return result

def condition_checker(M): ##Given a Assignment_datum_for_G 
    #We record the number that work and don't work
    N_that_work=[]
    N_that_do_not_work=[]

    for N in M:
        N=[(pair[0].copy(immutable=True),pair[1]) for pair in N]
        Assignments=get_assignment_ordered_for_wstab(N,ordering_trees)
        sigma=w_stability(G,Assignments)
        x,y=check_size(G,sigma)

        if x==True:
            N_that_work.append((Assignments,sigma))
        if x==False:
            N_that_do_not_work.append(Assignments)

    print(f"Of the {len(M)} assignment datums: \n {len(N_that_work)}: Produce a stability condition \n {len(N_that_do_not_work)}: Do NOT produce a stability condition")
    return N_that_work,N_that_do_not_work

In [30]:
# x=[analyse_specific_assignment,get_ordered_list,get_assignment_ordered_for_wstab,condition_checker]
# find_functions_used_l(x)

In [27]:
#Storing files 
def output_to_file(G,assignment_datum,ordering_trees):#in text files
    stabs=assignment_datum[0] #Those that work


    print(f"Information: Here are all stability conditions for the graph G")
    print(f"Graph: {G.edges()} \n")
    
    print(f"There are {len(stabs)} stability conditions for G. \n")
    print(f"The spanning trees of G are ordered as follows:")
    for index,tree in enumerate(ordering_trees):
        print(f"Tree {index+1}: {tree.edges()}" )

    print("\n We will now list all stabilty conditions. The data will be presented in two lines:")
    print("   1. The assignment datum on trees the order of which will be given by the Tree order,")
    print("   2. The union of sets of the form assignment datum of T + the set of break divisors for T. \n" )          

    for pair in stabs:
        print(pair[0]) #assignment datum
        #Put sigma to list format
        print(pair[1].tolist()) #lbm
        print("\n")
    
    return
def store_data(G,assignment_datum,ordering_trees,graphname="test"):
    name=f"{graphname}.txt"
    with open(name, "w") as f:
        sys.stdout = f

        output_to_file(G,assignment_datum,ordering_trees)

        sys.stdout = sys.__stdout__
    return

def pickle_assignment_datum(graphname,assignment_datum):
    # Pickle the list
    filename=f"{graphname}.pkl"
    data=assignment_datum[0]
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    return

def unpickle_assignment_datum(graphname):
    # Unpickle the list
    filename=f"{graphname}.pkl"
    with open(filename, 'rb') as f:
        unpickled_list = pickle.load(f)
    return unpickled_list

In [31]:
# x=[output_to_file,store_data,pickle_assignment_datum,unpickle_assignment_datum]
# find_functions_used_l(x)

# Used Throughout

In [2]:
import numpy as np
import networkx as nx
from collections import Counter
import itertools 
import random
from itertools import combinations, permutations
import sys
import pickle

In [3]:
import ast
import inspect

In [5]:
def find_functions_used(func):
    
    """Add print Objective."""
    
    functions_used = set()
    tree = ast.parse(inspect.getsource(func))
    for node in ast.walk(tree):
        if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            functions_used.add(node.func.id)
    return functions_used

def extract_objective(func):
    """
    Extracts the string of the line containing "Objective" from the docstring of a given function.

    Args:
        func: The function to extract the objective from.

    Returns:
        A string containing the objective.
    """
    
    """
        def f():

        
            Objective: This is a test
            Input:
            Return
        

        x=33

        return
        extract_objective(f)
    """
    docstring = inspect.getdoc(func)
    if docstring is None:
        return None
    lines = docstring.split('\n')
    for line in lines:
        if 'Objective' in line:
            return line.strip()
    return None


def find_functions_used_l(l,flag=False):
    for func in l:
        functions_used = find_functions_used(func)
        if flag==True:
            print(f"{func.__name__}:\n{extract_objective(func)}\n{functions_used} \n")
        if flag==False:
            print(f"{func.__name__}:\n{functions_used} \n")

In [ ]:
# l = [f1, f2, f3]
# find_functions_used_l(l)

# Leftovers